In [55]:
import pandas as pd
from openai import OpenAI
import pdfminer
from pdfminer.high_level import extract_text
import spacy
from collections import defaultdict
import re

In [25]:
print(pdfminer.__version__)


20231228


1. Find a PDF scraper to scrape CV of student
2. Obtain metadata (Student Profile) of the student from the csv and store it in a dataframe
3. Obtain data of jobs


## Create a database of jobs (Create using synthetic dataset) --> Input this into a VectorDB

Job ID, Job Title, Industry, Required Education, Required Degree Field, Years of Experience, Hard Skills, Soft Skills


## Create for Student (Use a PDFreader on a resume, then use a NER model to fill the table)
Name, Education Level, Degree Field, Name of University, GPA, Work Experience, Hard Skill, Soft Skills, Learning Speed (Categories like Fast Medium Slow)

### Next Steps:
Explore User-Item Matrix Creation (k-NearestNeighbour)
Explore LLM with RAG
Explore Matrix Factorisation (Collaborative Filteriing)
Explore GNNs (Graph-Based Recommendation Systems)


### Create the list of jobs and the skill sets required for each job

In [ ]:
list_jobs = [
    "Immigration Officer", "Judge", "Paralegal", "Legal Assistant", "Public Defender", "Urban Policy Planner",
    "Supply Chain Manager", "Logistics Coordinator", "Transportation Planner", "Warehouse Manager", "Freight Broker",
    "Truck Driver", "Airline Operations Manager", "Port Manager", "Inventory Manager", "Procurement Officer",
    "AI Product Manager", "Prompt Engineer (for AI)", "Data Privacy Officer", "Robotics Engineer", "VR/AR Developer",
    "Quantum Computing Researcher", "Drone Operator", "Space Tourism Guide", "Smart Home Technician", "Climate Tech Entrepreneur",
    "Blockchain Product Manager", "Autonomous Vehicle Engineer", "IoT (Internet of Things) Engineer", "Digital Twin Engineer", "Ethical Hacker"
]

prompt = f"""
You are a highly experienced headhunting specialist helping university students find suitable graduate jobs.

You are tasked to create a synthetic dataset for 200 unique jobs. 
Use the following list of jobs exactly as provided:
{list_jobs}

For each job, generate the following fields:
- Job ID: A unique integer starting from 1.
- Job Title: Exactly as given from the list (no changes).
- Industry: Choose appropriately from these: ["Technology", "Finance", "Healthcare", "Arts and Media", "Education", "Human Resources and Operations", "Construction and Engineering", "Environmental and Sustainability", "Travel, Hospitality and Tourism", "Science and Research", "Retail and Sales", "Law and Government", "Transportation and Logistics", "Emerging Technologies"].
- Required Education: Choose one of ["Bachelor", "Master", "None"].
- Required Degree Field: Choose one of ["Computing", "Information Systems", "Arts and Social Sciences", "Science", "Engineering", "Design", "Business", "Medicine (Medicine and Nursing Students)"].
- Years of Experience: Generate a realistic integer between 0 and 5.
- Hard Skills: Provide a Python list of 2-5 relevant technical skills as strings (e.g., ["Python", "Data Analysis", "Cloud Computing"]).
- Soft Skills: Provide a Python list of 2-5 soft skills as strings (e.g., ["Problem-Solving", "Communication", "Adaptability"]).

**Important formatting rules**:
- Output only the result as a **Python list of dictionaries**.
- Each dictionary represents one job.
- Do not add any explanation text. Only output the Python list object.
- Make sure the skills and degree fields make logical sense based on the job.

Example format for one job:

{{
  "Job ID": 1,
  "Job Title": "Software Engineer",
  "Industry": "Technology",
  "Required Education": "Bachelor",
  "Required Degree Field": "Computing",
  "Years of Experience": 2,
  "Hard Skills": ["Python", "Software Development", "Version Control"],
  "Soft Skills": ["Problem-Solving", "Teamwork"]
}}

You must Start from Job ID 170 and must continue until Job ID 200.
"""



client = OpenAI(
    api_key = "sk-06cb346d24fe4b36bd7b8b9fe91eaff0",
    base_url="https://api.deepseek.com")

response = client.chat.completions.create(
    model = "deepseek-chat",
    messages = [
        {"role": "system", "content": "You are an extremely experienced head hunting specialist."},
        {"role": "user", "content": prompt},
    ],
    stream = False
)

print(response.choices[0].message.content)


In [19]:
job_data = [
    {
        "Job ID": 1,
        "Job Title": "Software Engineer",
        "Industry": "Technology",
        "Required Education": "Bachelor",
        "Required Degree Field": "Computing",
        "Years of Experience": 2,
        "Hard Skills": ["Python", "Software Development", "Version Control"],
        "Soft Skills": ["Problem-Solving", "Teamwork"]
    },
    {
        "Job ID": 2,
        "Job Title": "Frontend Developer",
        "Industry": "Technology",
        "Required Education": "Bachelor",
        "Required Degree Field": "Computing",
        "Years of Experience": 1,
        "Hard Skills": ["JavaScript", "HTML/CSS", "React"],
        "Soft Skills": ["Creativity", "Attention to Detail"]
    },
    {
        "Job ID": 3,
        "Job Title": "Backend Developer",
        "Industry": "Technology",
        "Required Education": "Bachelor",
        "Required Degree Field": "Computing",
        "Years of Experience": 2,
        "Hard Skills": ["Java", "SQL", "API Development"],
        "Soft Skills": ["Logical Thinking", "Collaboration"]
    },
    {
        "Job ID": 4,
        "Job Title": "Full Stack Developer",
        "Industry": "Technology",
        "Required Education": "Bachelor",
        "Required Degree Field": "Computing",
        "Years of Experience": 3,
        "Hard Skills": ["JavaScript", "Python", "Node.js"],
        "Soft Skills": ["Adaptability", "Communication"]
    },
    {
        "Job ID": 5,
        "Job Title": "DevOps Engineer",
        "Industry": "Technology",
        "Required Education": "Bachelor",
        "Required Degree Field": "Computing",
        "Years of Experience": 3,
        "Hard Skills": ["Docker", "Kubernetes", "CI/CD"],
        "Soft Skills": ["Problem-Solving", "Teamwork"]
    },
    {
        "Job ID": 6,
        "Job Title": "Machine Learning Engineer",
        "Industry": "Technology",
        "Required Education": "Master",
        "Required Degree Field": "Computing",
        "Years of Experience": 2,
        "Hard Skills": ["Python", "TensorFlow", "Data Modeling"],
        "Soft Skills": ["Analytical Thinking", "Creativity"]
    },
    {
        "Job ID": 7,
        "Job Title": "Data Scientist",
        "Industry": "Technology",
        "Required Education": "Master",
        "Required Degree Field": "Computing",
        "Years of Experience": 2,
        "Hard Skills": ["Python", "Machine Learning", "Data Visualization"],
        "Soft Skills": ["Critical Thinking", "Communication"]
    },
    {
        "Job ID": 8,
        "Job Title": "Data Analyst",
        "Industry": "Technology",
        "Required Education": "Bachelor",
        "Required Degree Field": "Computing",
        "Years of Experience": 1,
        "Hard Skills": ["SQL", "Excel", "Tableau"],
        "Soft Skills": ["Attention to Detail", "Problem-Solving"]
    },
    {
        "Job ID": 9,
        "Job Title": "Data Engineer",
        "Industry": "Technology",
        "Required Education": "Bachelor",
        "Required Degree Field": "Computing",
        "Years of Experience": 2,
        "Hard Skills": ["SQL", "ETL", "Big Data"],
        "Soft Skills": ["Collaboration", "Logical Thinking"]
    },
    {
        "Job ID": 10,
        "Job Title": "Cloud Architect",
        "Industry": "Technology",
        "Required Education": "Master",
        "Required Degree Field": "Computing",
        "Years of Experience": 4,
        "Hard Skills": ["AWS", "Azure", "Cloud Security"],
        "Soft Skills": ["Strategic Thinking", "Leadership"]
    },
    {
        "Job ID": 11,
        "Job Title": "Cybersecurity Analyst",
        "Industry": "Technology",
        "Required Education": "Bachelor",
        "Required Degree Field": "Computing",
        "Years of Experience": 2,
        "Hard Skills": ["Network Security", "Ethical Hacking", "SIEM"],
        "Soft Skills": ["Attention to Detail", "Problem-Solving"]
    },
    {
        "Job ID": 12,
        "Job Title": "AI Researcher",
        "Industry": "Science and Research",
        "Required Education": "Master",
        "Required Degree Field": "Computing",
        "Years of Experience": 3,
        "Hard Skills": ["Python", "Deep Learning", "Natural Language Processing"],
        "Soft Skills": ["Innovation", "Critical Thinking"]
    },
    {
        "Job ID": 13,
        "Job Title": "Blockchain Developer",
        "Industry": "Emerging Technologies",
        "Required Education": "Bachelor",
        "Required Degree Field": "Computing",
        "Years of Experience": 2,
        "Hard Skills": ["Solidity", "Smart Contracts", "Ethereum"],
        "Soft Skills": ["Problem-Solving", "Collaboration"]
    },
    {
        "Job ID": 14,
        "Job Title": "Web Developer",
        "Industry": "Technology",
        "Required Education": "Bachelor",
        "Required Degree Field": "Computing",
        "Years of Experience": 1,
        "Hard Skills": ["HTML/CSS", "JavaScript", "PHP"],
        "Soft Skills": ["Creativity", "Adaptability"]
    },
    {
        "Job ID": 15,
        "Job Title": "Game Developer",
        "Industry": "Arts and Media",
        "Required Education": "Bachelor",
        "Required Degree Field": "Computing",
        "Years of Experience": 2,
        "Hard Skills": ["Unity", "C#", "3D Modeling"],
        "Soft Skills": ["Creativity", "Teamwork"]
    },
    {
        "Job ID": 16,
        "Job Title": "UX Designer",
        "Industry": "Technology",
        "Required Education": "Bachelor",
        "Required Degree Field": "Design",
        "Years of Experience": 2,
        "Hard Skills": ["Figma", "User Research", "Wireframing"],
        "Soft Skills": ["Empathy", "Communication"]
    },
    {
        "Job ID": 17,
        "Job Title": "UI Designer",
        "Industry": "Technology",
        "Required Education": "Bachelor",
        "Required Degree Field": "Design",
        "Years of Experience": 1,
        "Hard Skills": ["Adobe XD", "Typography", "Color Theory"],
        "Soft Skills": ["Creativity", "Attention to Detail"]
    },
    {
        "Job ID": 18,
        "Job Title": "Product Manager (Tech)",
        "Industry": "Technology",
        "Required Education": "Master",
        "Required Degree Field": "Business",
        "Years of Experience": 4,
        "Hard Skills": ["Agile Methodology", "Product Roadmapping", "Market Research"],
        "Soft Skills": ["Leadership", "Communication"]
    },
    {
        "Job ID": 19,
        "Job Title": "Mobile App Developer",
        "Industry": "Technology",
        "Required Education": "Bachelor",
        "Required Degree Field": "Computing",
        "Years of Experience": 2,
        "Hard Skills": ["Swift", "Kotlin", "React Native"],
        "Soft Skills": ["Problem-Solving", "Adaptability"]
    },
    {
        "Job ID": 20,
        "Job Title": "Embedded Systems Engineer",
        "Industry": "Technology",
        "Required Education": "Bachelor",
        "Required Degree Field": "Engineering",
        "Years of Experience": 3,
        "Hard Skills": ["C/C++", "Microcontrollers", "RTOS"],
        "Soft Skills": ["Attention to Detail", "Analytical Thinking"]
    },
    {
        "Job ID": 21,
        "Job Title": "Financial Analyst",
        "Industry": "Finance",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 2,
        "Hard Skills": ["Excel", "Financial Modeling", "Data Analysis"],
        "Soft Skills": ["Analytical Thinking", "Communication"]
    },
    {
        "Job ID": 22,
        "Job Title": "Investment Banker",
        "Industry": "Finance",
        "Required Education": "Master",
        "Required Degree Field": "Business",
        "Years of Experience": 3,
        "Hard Skills": ["Financial Modeling", "Valuation", "M&A"],
        "Soft Skills": ["Negotiation", "Attention to Detail"]
    },
    {
        "Job ID": 23,
        "Job Title": "Management Consultant",
        "Industry": "Finance",
        "Required Education": "Master",
        "Required Degree Field": "Business",
        "Years of Experience": 3,
        "Hard Skills": ["Strategic Planning", "Market Analysis", "Project Management"],
        "Soft Skills": ["Problem-Solving", "Communication"]
    },
    {
        "Job ID": 24,
        "Job Title": "Auditor",
        "Industry": "Finance",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 2,
        "Hard Skills": ["Accounting", "Risk Assessment", "Audit Software"],
        "Soft Skills": ["Attention to Detail", "Integrity"]
    },
    {
        "Job ID": 25,
        "Job Title": "Accountant",
        "Industry": "Finance",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 1,
        "Hard Skills": ["Bookkeeping", "Tax Preparation", "Financial Reporting"],
        "Soft Skills": ["Accuracy", "Organization"]
    },
    {
        "Job ID": 26,
        "Job Title": "Risk Analyst",
        "Industry": "Finance",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 2,
        "Hard Skills": ["Risk Modeling", "Data Analysis", "Statistics"],
        "Soft Skills": ["Critical Thinking", "Attention to Detail"]
    },
    {
        "Job ID": 27,
        "Job Title": "Actuary",
        "Industry": "Finance",
        "Required Education": "Master",
        "Required Degree Field": "Science",
        "Years of Experience": 3,
        "Hard Skills": ["Statistical Modeling", "Probability", "Financial Mathematics"],
        "Soft Skills": ["Analytical Thinking", "Problem-Solving"]
    },
    {
        "Job ID": 28,
        "Job Title": "Tax Consultant",
        "Industry": "Finance",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 2,
        "Hard Skills": ["Tax Law", "Financial Planning", "Compliance"],
        "Soft Skills": ["Communication", "Attention to Detail"]
    },
    {
        "Job ID": 29,
        "Job Title": "Corporate Lawyer",
        "Industry": "Law and Government",
        "Required Education": "Master",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 4,
        "Hard Skills": ["Contract Law", "Corporate Governance", "Legal Research"],
        "Soft Skills": ["Negotiation", "Critical Thinking"]
    },
    {
        "Job ID": 30,
        "Job Title": "Compliance Officer",
        "Industry": "Law and Government",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 2,
        "Hard Skills": ["Regulatory Compliance", "Risk Management", "Auditing"],
        "Soft Skills": ["Attention to Detail", "Integrity"]
    },
    {
        "Job ID": 31,
        "Job Title": "Business Development Manager",
        "Industry": "Business",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 3,
        "Hard Skills": ["Market Research", "Sales Strategies", "CRM"],
        "Soft Skills": ["Negotiation", "Communication"]
    },
    {
        "Job ID": 32,
        "Job Title": "Marketing Analyst",
        "Industry": "Retail and Sales",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 1,
        "Hard Skills": ["Data Analysis", "SEO", "Google Analytics"],
        "Soft Skills": ["Creativity", "Communication"]
    },
    {
        "Job ID": 33,
        "Job Title": "Project Manager",
        "Industry": "Business",
        "Required Education": "Master",
        "Required Degree Field": "Business",
        "Years of Experience": 3,
        "Hard Skills": ["Agile Methodology", "Budgeting", "Stakeholder Management"],
        "Soft Skills": ["Leadership", "Organization"]
    },
    {
        "Job ID": 34,
        "Job Title": "Venture Capital Analyst",
        "Industry": "Finance",
        "Required Education": "Master",
        "Required Degree Field": "Business",
        "Years of Experience": 2,
        "Hard Skills": ["Financial Modeling", "Due Diligence", "Market Analysis"],
        "Soft Skills": ["Analytical Thinking", "Communication"]
    },
    {
        "Job ID": 35,
        "Job Title": "Fund Manager",
        "Industry": "Finance",
        "Required Education": "Master",
        "Required Degree Field": "Business",
        "Years of Experience": 5,
        "Hard Skills": ["Portfolio Management", "Investment Strategies", "Risk Assessment"],
        "Soft Skills": ["Decision-Making", "Leadership"]
    },
    {
        "Job ID": 36,
        "Job Title": "Sales Executive",
        "Industry": "Retail and Sales",
        "Required Education": "None",
        "Required Degree Field": "Business",
        "Years of Experience": 1,
        "Hard Skills": ["CRM", "Negotiation", "Market Research"],
        "Soft Skills": ["Communication", "Persuasion"]
    },
    {
        "Job ID": 37,
        "Job Title": "Entrepreneur / Startup Founder",
        "Industry": "Business",
        "Required Education": "None",
        "Required Degree Field": "Business",
        "Years of Experience": 3,
        "Hard Skills": ["Business Planning", "Fundraising", "Product Development"],
        "Soft Skills": ["Leadership", "Risk-Taking"]
    },
    {
        "Job ID": 38,
        "Job Title": "Strategy Consultant",
        "Industry": "Business",
        "Required Education": "Master",
        "Required Degree Field": "Business",
        "Years of Experience": 4,
        "Hard Skills": ["Market Analysis", "Competitive Research", "Business Modeling"],
        "Soft Skills": ["Critical Thinking", "Communication"]
    },
    {
        "Job ID": 39,
        "Job Title": "Business Analyst",
        "Industry": "Business",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 2,
        "Hard Skills": ["Data Analysis", "Process Improvement", "Requirement Gathering"],
        "Soft Skills": ["Problem-Solving", "Communication"]
    },
    {
        "Job ID": 40,
        "Job Title": "Real Estate Analyst",
        "Industry": "Finance",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 2,
        "Hard Skills": ["Market Research", "Financial Modeling", "Property Valuation"],
        "Soft Skills": ["Analytical Thinking", "Attention to Detail"]
    },
    {
        "Job ID": 41,
        "Job Title": "Doctor (General Practitioner)",
        "Industry": "Healthcare",
        "Required Education": "Master",
        "Required Degree Field": "Medicine (Medicine and Nursing Students)",
        "Years of Experience": 3,
        "Hard Skills": ["Diagnosis", "Patient Care", "Medical Knowledge"],
        "Soft Skills": ["Empathy", "Communication"]
    },
    {
        "Job ID": 42,
        "Job Title": "Surgeon",
        "Industry": "Healthcare",
        "Required Education": "Master",
        "Required Degree Field": "Medicine (Medicine and Nursing Students)",
        "Years of Experience": 5,
        "Hard Skills": ["Surgical Procedures", "Anatomy", "Patient Management"],
        "Soft Skills": ["Precision", "Stress Management"]
    },
    {
        "Job ID": 43,
        "Job Title": "Nurse",
        "Industry": "Healthcare",
        "Required Education": "Bachelor",
        "Required Degree Field": "Medicine (Medicine and Nursing Students)",
        "Years of Experience": 2,
        "Hard Skills": ["Patient Care", "Medication Administration", "First Aid"],
        "Soft Skills": ["Compassion", "Teamwork"]
    },
    {
        "Job ID": 44,
        "Job Title": "Dentist",
        "Industry": "Healthcare",
        "Required Education": "Master",
        "Required Degree Field": "Medicine (Medicine and Nursing Students)",
        "Years of Experience": 3,
        "Hard Skills": ["Dental Procedures", "Oral Health", "Patient Education"],
        "Soft Skills": ["Manual Dexterity", "Communication"]
    },
    {
        "Job ID": 45,
        "Job Title": "Pharmacist",
        "Industry": "Healthcare",
        "Required Education": "Master",
        "Required Degree Field": "Medicine (Medicine and Nursing Students)",
        "Years of Experience": 2,
        "Hard Skills": ["Medication Management", "Pharmaceutical Knowledge", "Dosage Calculation"],
        "Soft Skills": ["Attention to Detail", "Customer Service"]
    },
    {
        "Job ID": 46,
        "Job Title": "Physiotherapist",
        "Industry": "Healthcare",
        "Required Education": "Bachelor",
        "Required Degree Field": "Medicine (Medicine and Nursing Students)",
        "Years of Experience": 2,
        "Hard Skills": ["Rehabilitation Techniques", "Exercise Prescription", "Patient Assessment"],
        "Soft Skills": ["Empathy", "Patience"]
    },
    {
        "Job ID": 47,
        "Job Title": "Radiologist",
        "Industry": "Healthcare",
        "Required Education": "Master",
        "Required Degree Field": "Medicine (Medicine and Nursing Students)",
        "Years of Experience": 4,
        "Hard Skills": ["Medical Imaging", "Diagnosis", "Radiation Safety"],
        "Soft Skills": ["Analytical Thinking", "Attention to Detail"]
    },
    {
        "Job ID": 48,
        "Job Title": "Occupational Therapist",
        "Industry": "Healthcare",
        "Required Education": "Master",
        "Required Degree Field": "Medicine (Medicine and Nursing Students)",
        "Years of Experience": 3,
        "Hard Skills": ["Patient Assessment", "Rehabilitation Techniques", "Therapeutic Equipment"],
        "Soft Skills": ["Empathy", "Communication", "Patience"]
    },
    {
        "Job ID": 49,
        "Job Title": "Clinical Research Associate",
        "Industry": "Healthcare",
        "Required Education": "Bachelor",
        "Required Degree Field": "Science",
        "Years of Experience": 2,
        "Hard Skills": ["Clinical Trials", "Data Collection", "Regulatory Compliance"],
        "Soft Skills": ["Attention to Detail", "Organization", "Teamwork"]
    },
    {
        "Job ID": 50,
        "Job Title": "Medical Laboratory Technologist",
        "Industry": "Healthcare",
        "Required Education": "Bachelor",
        "Required Degree Field": "Science",
        "Years of Experience": 1,
        "Hard Skills": ["Lab Testing", "Microscopy", "Data Analysis"],
        "Soft Skills": ["Precision", "Time Management", "Collaboration"]
    },
    {
        "Job ID": 51,
        "Job Title": "Paramedic",
        "Industry": "Healthcare",
        "Required Education": "Bachelor",
        "Required Degree Field": "Medicine (Medicine and Nursing Students)",
        "Years of Experience": 2,
        "Hard Skills": ["Emergency Care", "CPR", "Medical Equipment"],
        "Soft Skills": ["Calm Under Pressure", "Quick Decision-Making", "Compassion"]
    },
    {
        "Job ID": 52,
        "Job Title": "Psychiatrist",
        "Industry": "Healthcare",
        "Required Education": "Master",
        "Required Degree Field": "Medicine (Medicine and Nursing Students)",
        "Years of Experience": 5,
        "Hard Skills": ["Diagnosis", "Psychotherapy", "Medication Management"],
        "Soft Skills": ["Active Listening", "Empathy", "Patience"]
    },
    {
        "Job ID": 53,
        "Job Title": "Psychologist",
        "Industry": "Healthcare",
        "Required Education": "Master",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 3,
        "Hard Skills": ["Psychological Testing", "Counseling", "Behavioral Analysis"],
        "Soft Skills": ["Empathy", "Communication", "Critical Thinking"]
    },
    {
        "Job ID": 54,
        "Job Title": "Medical Sales Representative",
        "Industry": "Healthcare",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 2,
        "Hard Skills": ["Sales Techniques", "Product Knowledge", "CRM Software"],
        "Soft Skills": ["Negotiation", "Communication", "Persuasion"]
    },
    {
        "Job ID": 55,
        "Job Title": "Healthcare Administrator",
        "Industry": "Healthcare",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 4,
        "Hard Skills": ["Healthcare Regulations", "Budget Management", "Staff Scheduling"],
        "Soft Skills": ["Leadership", "Organization", "Problem-Solving"]
    },
    {
        "Job ID": 56,
        "Job Title": "Veterinary Surgeon",
        "Industry": "Healthcare",
        "Required Education": "Master",
        "Required Degree Field": "Medicine (Medicine and Nursing Students)",
        "Years of Experience": 3,
        "Hard Skills": ["Surgical Procedures", "Animal Diagnosis", "Anesthesia"],
        "Soft Skills": ["Compassion", "Attention to Detail", "Patience"]
    },
    {
        "Job ID": 57,
        "Job Title": "Nutritionist",
        "Industry": "Healthcare",
        "Required Education": "Bachelor",
        "Required Degree Field": "Science",
        "Years of Experience": 1,
        "Hard Skills": ["Diet Planning", "Nutritional Assessment", "Food Science"],
        "Soft Skills": ["Communication", "Empathy", "Motivation"]
    },
    {
        "Job ID": 58,
        "Job Title": "Optometrist",
        "Industry": "Healthcare",
        "Required Education": "Master",
        "Required Degree Field": "Medicine (Medicine and Nursing Students)",
        "Years of Experience": 2,
        "Hard Skills": ["Eye Examinations", "Contact Lens Fitting", "Vision Therapy"],
        "Soft Skills": ["Patient Care", "Attention to Detail", "Communication"]
    },
    {
        "Job ID": 59,
        "Job Title": "Public Health Officer",
        "Industry": "Healthcare",
        "Required Education": "Master",
        "Required Degree Field": "Science",
        "Years of Experience": 4,
        "Hard Skills": ["Epidemiology", "Health Policy", "Disease Prevention"],
        "Soft Skills": ["Leadership", "Communication", "Critical Thinking"]
    },
    {
        "Job ID": 60,
        "Job Title": "Medical Technologist",
        "Industry": "Healthcare",
        "Required Education": "Bachelor",
        "Required Degree Field": "Science",
        "Years of Experience": 2,
        "Hard Skills": ["Lab Testing", "Diagnostic Equipment", "Data Analysis"],
        "Soft Skills": ["Attention to Detail", "Teamwork", "Problem-Solving"]
    },
    {
        "Job ID": 61,
        "Job Title": "Graphic Designer",
        "Industry": "Arts and Media",
        "Required Education": "Bachelor",
        "Required Degree Field": "Design",
        "Years of Experience": 1,
        "Hard Skills": ["Adobe Creative Suite", "Typography", "Branding"],
        "Soft Skills": ["Creativity", "Communication", "Time Management"]
    },
    {
        "Job ID": 62,
        "Job Title": "Animator",
        "Industry": "Arts and Media",
        "Required Education": "Bachelor",
        "Required Degree Field": "Design",
        "Years of Experience": 2,
        "Hard Skills": ["2D/3D Animation", "Storyboarding", "Motion Graphics"],
        "Soft Skills": ["Creativity", "Collaboration", "Attention to Detail"]
    },
    {
        "Job ID": 63,
        "Job Title": "Illustrator",
        "Industry": "Arts and Media",
        "Required Education": "Bachelor",
        "Required Degree Field": "Design",
        "Years of Experience": 1,
        "Hard Skills": ["Digital Illustration", "Sketching", "Concept Art"],
        "Soft Skills": ["Creativity", "Adaptability", "Attention to Detail"]
    },
    {
        "Job ID": 64,
        "Job Title": "Video Editor",
        "Industry": "Arts and Media",
        "Required Education": "Bachelor",
        "Required Degree Field": "Design",
        "Years of Experience": 2,
        "Hard Skills": ["Adobe Premiere", "Color Grading", "Sound Editing"],
        "Soft Skills": ["Creativity", "Time Management", "Collaboration"]
    },
    {
        "Job ID": 65,
        "Job Title": "Film Director",
        "Industry": "Arts and Media",
        "Required Education": "Bachelor",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 5,
        "Hard Skills": ["Storytelling", "Cinematography", "Script Analysis"],
        "Soft Skills": ["Leadership", "Creativity", "Communication"]
    },
    {
        "Job ID": 66,
        "Job Title": "Music Producer",
        "Industry": "Arts and Media",
        "Required Education": "Bachelor",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 3,
        "Hard Skills": ["Audio Mixing", "MIDI Programming", "Sound Design"],
        "Soft Skills": ["Creativity", "Collaboration", "Attention to Detail"]
    },
    {
        "Job ID": 67,
        "Job Title": "Sound Engineer",
        "Industry": "Arts and Media",
        "Required Education": "Bachelor",
        "Required Degree Field": "Engineering",
        "Years of Experience": 2,
        "Hard Skills": ["Audio Editing", "Live Sound Mixing", "Acoustics"],
        "Soft Skills": ["Attention to Detail", "Problem-Solving", "Teamwork"]
    },
    {
        "Job ID": 68,
        "Job Title": "Photographer",
        "Industry": "Arts and Media",
        "Required Education": "Bachelor",
        "Required Degree Field": "Design",
        "Years of Experience": 1,
        "Hard Skills": ["Photo Editing", "Lighting Techniques", "Composition"],
        "Soft Skills": ["Creativity", "Attention to Detail", "Adaptability"]
    },
    {
        "Job ID": 69,
        "Job Title": "Content Creator",
        "Industry": "Arts and Media",
        "Required Education": "Bachelor",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 1,
        "Hard Skills": ["Video Production", "Social Media Management", "Copywriting"],
        "Soft Skills": ["Creativity", "Communication", "Adaptability"]
    },
    {
        "Job ID": 70,
        "Job Title": "Copywriter",
        "Industry": "Arts and Media",
        "Required Education": "Bachelor",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 2,
        "Hard Skills": ["SEO Writing", "Brand Messaging", "Content Strategy"],
        "Soft Skills": ["Creativity", "Communication", "Research"]
    },
    {
        "Job ID": 71,
        "Job Title": "Journalist",
        "Industry": "Arts and Media",
        "Required Education": "Bachelor",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 2,
        "Hard Skills": ["Investigative Reporting", "Interviewing", "Fact-Checking"],
        "Soft Skills": ["Communication", "Curiosity", "Adaptability"]
    },
    {
        "Job ID": 72,
        "Job Title": "Screenwriter",
        "Industry": "Arts and Media",
        "Required Education": "Bachelor",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 3,
        "Hard Skills": ["Scriptwriting", "Story Development", "Dialogue Writing"],
        "Soft Skills": ["Creativity", "Persistence", "Collaboration"]
    },
    {
        "Job ID": 73,
        "Job Title": "Social Media Manager",
        "Industry": "Arts and Media",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 2,
        "Hard Skills": ["Content Scheduling", "Analytics", "Campaign Management"],
        "Soft Skills": ["Creativity", "Communication", "Adaptability"]
    },
    {
        "Job ID": 74,
        "Job Title": "Brand Strategist",
        "Industry": "Arts and Media",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 3,
        "Hard Skills": ["Market Research", "Brand Positioning", "Consumer Insights"],
        "Soft Skills": ["Strategic Thinking", "Communication", "Creativity"]
    },
    {
        "Job ID": 75,
        "Job Title": "Art Director",
        "Industry": "Arts and Media",
        "Required Education": "Bachelor",
        "Required Degree Field": "Design",
        "Years of Experience": 4,
        "Hard Skills": ["Visual Storytelling", "Creative Direction", "Branding"],
        "Soft Skills": ["Leadership", "Creativity", "Collaboration"]
    },
    {
        "Job ID": 76,
        "Job Title": "Creative Director",
        "Industry": "Arts and Media",
        "Required Education": "Bachelor",
        "Required Degree Field": "Design",
        "Years of Experience": 5,
        "Hard Skills": ["Campaign Development", "Creative Strategy", "Team Leadership"],
        "Soft Skills": ["Visionary Thinking", "Leadership", "Communication"]
    },
    {
        "Job ID": 77,
        "Job Title": "Game Designer",
        "Industry": "Arts and Media",
        "Required Education": "Bachelor",
        "Required Degree Field": "Design",
        "Years of Experience": 2,
        "Hard Skills": ["Game Mechanics", "Level Design", "Prototyping"],
        "Soft Skills": ["Creativity", "Problem-Solving", "Collaboration"]
    },
    {
        "Job ID": 78,
        "Job Title": "Fashion Designer",
        "Industry": "Arts and Media",
        "Required Education": "Bachelor",
        "Required Degree Field": "Design",
        "Years of Experience": 3,
        "Hard Skills": ["Pattern Making", "Textile Knowledge", "Fashion Illustration"],
        "Soft Skills": ["Creativity", "Attention to Detail", "Trend Awareness"]
    },
    {
        "Job ID": 79,
        "Job Title": "Interior Designer",
        "Industry": "Arts and Media",
        "Required Education": "Bachelor",
        "Required Degree Field": "Design",
        "Years of Experience": 2,
        "Hard Skills": ["Space Planning", "3D Rendering", "Material Selection"],
        "Soft Skills": ["Creativity", "Communication", "Problem-Solving"]
    },
    {
        "Job ID": 80,
        "Job Title": "Web Content Manager",
        "Industry": "Technology",
        "Required Education": "Bachelor",
        "Required Degree Field": "Computing",
        "Years of Experience": 2,
        "Hard Skills": ["Content Management Systems", "SEO", "HTML/CSS"],
        "Soft Skills": ["Communication", "Organization", "Attention to Detail"]
    },
    {
        "Job ID": 81,
        "Job Title": "University Lecturer",
        "Industry": "Education",
        "Required Education": "Master",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 4,
        "Hard Skills": ["Curriculum Development", "Academic Research", "Lecture Delivery"],
        "Soft Skills": ["Communication", "Mentorship", "Public Speaking"]
    },
    {
        "Job ID": 82,
        "Job Title": "Primary School Teacher",
        "Industry": "Education",
        "Required Education": "Bachelor",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 2,
        "Hard Skills": ["Lesson Planning", "Classroom Management", "Child Development"],
        "Soft Skills": ["Patience", "Communication", "Empathy"]
    },
    {
        "Job ID": 83,
        "Job Title": "Secondary School Teacher",
        "Industry": "Education",
        "Required Education": "Bachelor",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 3,
        "Hard Skills": ["Subject Expertise", "Assessment Design", "Classroom Management"],
        "Soft Skills": ["Communication", "Patience", "Leadership"]
    },
    {
        "Job ID": 84,
        "Job Title": "Curriculum Developer",
        "Industry": "Education",
        "Required Education": "Master",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 3,
        "Hard Skills": ["Educational Standards", "Instructional Design", "Assessment Strategies"],
        "Soft Skills": ["Collaboration", "Creativity", "Problem-Solving"]
    },
    {
        "Job ID": 85,
        "Job Title": "Research Scientist",
        "Industry": "Science and Research",
        "Required Education": "Master",
        "Required Degree Field": "Science",
        "Years of Experience": 4,
        "Hard Skills": ["Experimental Design", "Data Analysis", "Scientific Writing"],
        "Soft Skills": ["Critical Thinking", "Curiosity", "Persistence"]
    },
    {
        "Job ID": 86,
        "Job Title": "Librarian",
        "Industry": "Education",
        "Required Education": "Master",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 2,
        "Hard Skills": ["Cataloging", "Information Retrieval", "Digital Archives"],
        "Soft Skills": ["Organization", "Customer Service", "Attention to Detail"]
    },
    {
        "Job ID": 87,
        "Job Title": "Instructional Designer",
        "Industry": "Education",
        "Required Education": "Master",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 3,
        "Hard Skills": ["E-Learning Development", "Learning Management Systems", "Assessment Design"],
        "Soft Skills": ["Collaboration", "Creativity", "Problem-Solving"]
    },
    {
        "Job ID": 88,
        "Job Title": "Special Education Teacher",
        "Industry": "Education",
        "Required Education": "Bachelor",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 3,
        "Hard Skills": ["Classroom Management", "Individualized Education Plans", "Behavioral Interventions"],
        "Soft Skills": ["Patience", "Empathy", "Communication"]
    },
    {
        "Job ID": 89,
        "Job Title": "Academic Advisor",
        "Industry": "Education",
        "Required Education": "Master",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 2,
        "Hard Skills": ["Academic Planning", "Student Counseling", "Degree Audits"],
        "Soft Skills": ["Active Listening", "Interpersonal Skills", "Problem-Solving"]
    },
    {
        "Job ID": 90,
        "Job Title": "Training Specialist",
        "Industry": "Human Resources and Operations",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 4,
        "Hard Skills": ["Curriculum Development", "Learning Management Systems", "Training Delivery"],
        "Soft Skills": ["Presentation Skills", "Adaptability", "Collaboration"]
    },
    {
        "Job ID": 91,
        "Job Title": "Learning and Development Manager",
        "Industry": "Human Resources and Operations",
        "Required Education": "Master",
        "Required Degree Field": "Business",
        "Years of Experience": 5,
        "Hard Skills": ["Training Strategy", "Performance Metrics", "E-Learning Tools"],
        "Soft Skills": ["Leadership", "Strategic Thinking", "Communication"]
    },
    {
        "Job ID": 92,
        "Job Title": "E-learning Specialist",
        "Industry": "Education",
        "Required Education": "Bachelor",
        "Required Degree Field": "Information Systems",
        "Years of Experience": 3,
        "Hard Skills": ["Learning Management Systems", "Instructional Design", "Multimedia Development"],
        "Soft Skills": ["Creativity", "Attention to Detail", "Problem-Solving"]
    },
    {
        "Job ID": 93,
        "Job Title": "Test Developer",
        "Industry": "Education",
        "Required Education": "Bachelor",
        "Required Degree Field": "Science",
        "Years of Experience": 2,
        "Hard Skills": ["Assessment Design", "Psychometrics", "Data Analysis"],
        "Soft Skills": ["Analytical Thinking", "Attention to Detail", "Collaboration"]
    },
    {
        "Job ID": 94,
        "Job Title": "Science Educator",
        "Industry": "Education",
        "Required Education": "Bachelor",
        "Required Degree Field": "Science",
        "Years of Experience": 1,
        "Hard Skills": ["Lesson Planning", "Laboratory Management", "STEM Education"],
        "Soft Skills": ["Enthusiasm", "Communication", "Patience"]
    },
    {
        "Job ID": 95,
        "Job Title": "Language Instructor",
        "Industry": "Education",
        "Required Education": "Bachelor",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 2,
        "Hard Skills": ["Language Proficiency", "Curriculum Development", "Cultural Knowledge"],
        "Soft Skills": ["Patience", "Communication", "Adaptability"]
    },
    {
        "Job ID": 96,
        "Job Title": "College Admissions Officer",
        "Industry": "Education",
        "Required Education": "Bachelor",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 3,
        "Hard Skills": ["Application Review", "Recruitment Strategies", "CRM Software"],
        "Soft Skills": ["Interpersonal Skills", "Decision-Making", "Organization"]
    },
    {
        "Job ID": 97,
        "Job Title": "Early Childhood Educator",
        "Industry": "Education",
        "Required Education": "Bachelor",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 2,
        "Hard Skills": ["Child Development", "Classroom Management", "Early Literacy"],
        "Soft Skills": ["Patience", "Creativity", "Empathy"]
    },
    {
        "Job ID": 98,
        "Job Title": "Education Policy Analyst",
        "Industry": "Education",
        "Required Education": "Master",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 4,
        "Hard Skills": ["Policy Research", "Data Analysis", "Report Writing"],
        "Soft Skills": ["Critical Thinking", "Communication", "Problem-Solving"]
    },
    {
        "Job ID": 99,
        "Job Title": "Tutor",
        "Industry": "Education",
        "Required Education": "None",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 1,
        "Hard Skills": ["Subject Matter Expertise", "Teaching Techniques", "Assessment"],
        "Soft Skills": ["Patience", "Communication", "Adaptability"]
    },
    {
        "Job ID": 100,
        "Job Title": "Career Counselor",
        "Industry": "Human Resources and Operations",
        "Required Education": "Master",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 3,
        "Hard Skills": ["Career Assessments", "Resume Writing", "Interview Coaching"],
        "Soft Skills": ["Empathy", "Active Listening", "Interpersonal Skills"]
    },
    {
        "Job ID": 101,
        "Job Title": "Human Resources Manager",
        "Industry": "Human Resources and Operations",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 5,
        "Hard Skills": ["Employee Relations", "HR Policies", "Performance Management"],
        "Soft Skills": ["Leadership", "Conflict Resolution", "Communication"]
    },
    {
        "Job ID": 102,
        "Job Title": "Recruiter",
        "Industry": "Human Resources and Operations",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 2,
        "Hard Skills": ["Talent Sourcing", "Applicant Tracking Systems", "Interviewing"],
        "Soft Skills": ["Networking", "Communication", "Persuasion"]
    },
    {
        "Job ID": 103,
        "Job Title": "Talent Acquisition Specialist",
        "Industry": "Human Resources and Operations",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 3,
        "Hard Skills": ["Recruitment Strategies", "Employer Branding", "Candidate Assessment"],
        "Soft Skills": ["Relationship Building", "Negotiation", "Communication"]
    },
    {
        "Job ID": 104,
        "Job Title": "Compensation and Benefits Manager",
        "Industry": "Human Resources and Operations",
        "Required Education": "Master",
        "Required Degree Field": "Business",
        "Years of Experience": 5,
        "Hard Skills": ["Salary Benchmarking", "Benefits Administration", "Payroll Systems"],
        "Soft Skills": ["Analytical Thinking", "Attention to Detail", "Communication"]
    },
    {
        "Job ID": 105,
        "Job Title": "Organizational Development Consultant",
        "Industry": "Human Resources and Operations",
        "Required Education": "Master",
        "Required Degree Field": "Business",
        "Years of Experience": 4,
        "Hard Skills": ["Change Management", "Leadership Training", "Employee Engagement"],
        "Soft Skills": ["Strategic Thinking", "Problem-Solving", "Interpersonal Skills"]
    },
    {
        "Job ID": 106,
        "Job Title": "Office Manager",
        "Industry": "Human Resources and Operations",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 3,
        "Hard Skills": ["Office Administration", "Budget Management", "Scheduling"],
        "Soft Skills": ["Organization", "Communication", "Multitasking"]
    },
    {
        "Job ID": 107,
        "Job Title": "Executive Assistant",
        "Industry": "Human Resources and Operations",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 2,
        "Hard Skills": ["Calendar Management", "Travel Coordination", "Document Preparation"],
        "Soft Skills": ["Discretion", "Time Management", "Communication"]
    },
    {
        "Job ID": 108,
        "Job Title": "Administrative Assistant",
        "Industry": "Human Resources and Operations",
        "Required Education": "None",
        "Required Degree Field": "Business",
        "Years of Experience": 1,
        "Hard Skills": ["Data Entry", "Filing Systems", "Office Software"],
        "Soft Skills": ["Organization", "Communication", "Attention to Detail"]
    },
    {
        "Job ID": 109,
        "Job Title": "Facilities Manager",
        "Industry": "Human Resources and Operations",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 4,
        "Hard Skills": ["Maintenance Coordination", "Space Planning", "Safety Compliance"],
        "Soft Skills": ["Problem-Solving", "Leadership", "Communication"]
    },
    {
        "Job ID": 110,
        "Job Title": "Event Coordinator",
        "Industry": "Travel, Hospitality and Tourism",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 2,
        "Hard Skills": ["Event Planning", "Vendor Management", "Budgeting"],
        "Soft Skills": ["Organization", "Creativity", "Interpersonal Skills"]
    },
    {
        "Job ID": 111,
        "Job Title": "Civil Engineer",
        "Industry": "Construction and Engineering",
        "Required Education": "Bachelor",
        "Required Degree Field": "Engineering",
        "Years of Experience": 3,
        "Hard Skills": ["Structural Design", "AutoCAD", "Project Management"],
        "Soft Skills": ["Problem-Solving", "Teamwork", "Attention to Detail"]
    },
    {
        "Job ID": 112,
        "Job Title": "Electrical Engineer",
        "Industry": "Construction and Engineering",
        "Required Education": "Bachelor",
        "Required Degree Field": "Engineering",
        "Years of Experience": 2,
        "Hard Skills": ["Circuit Design", "Power Systems", "MATLAB"],
        "Soft Skills": ["Analytical Thinking", "Collaboration", "Creativity"]
    },
    {
        "Job ID": 113,
        "Job Title": "Mechanical Engineer",
        "Industry": "Construction and Engineering",
        "Required Education": "Bachelor",
        "Required Degree Field": "Engineering",
        "Years of Experience": 3,
        "Hard Skills": ["CAD Modeling", "Thermodynamics", "Prototyping"],
        "Soft Skills": ["Problem-Solving", "Innovation", "Teamwork"]
    },
    {
        "Job ID": 114,
        "Job Title": "Structural Engineer",
        "Industry": "Construction and Engineering",
        "Required Education": "Master",
        "Required Degree Field": "Engineering",
        "Years of Experience": 4,
        "Hard Skills": ["Finite Element Analysis", "Seismic Design", "Steel and Concrete Design"],
        "Soft Skills": ["Attention to Detail", "Critical Thinking", "Collaboration"]
    },
    {
        "Job ID": 115,
        "Job Title": "Architect",
        "Industry": "Construction and Engineering",
        "Required Education": "Master",
        "Required Degree Field": "Design",
        "Years of Experience": 5,
        "Hard Skills": ["Architectural Design", "Revit", "Building Codes"],
        "Soft Skills": ["Creativity", "Visualization", "Communication"]
    },
    {
        "Job ID": 116,
        "Job Title": "Construction Project Manager",
        "Industry": "Construction and Engineering",
        "Required Education": "Bachelor",
        "Required Degree Field": "Engineering",
        "Years of Experience": 5,
        "Hard Skills": ["Project Scheduling", "Cost Estimation", "Contract Management"],
        "Soft Skills": ["Leadership", "Negotiation", "Problem-Solving"]
    },
    {
        "Job ID": 117,
        "Job Title": "Urban Planner",
        "Industry": "Construction and Engineering",
        "Required Education": "Master",
        "Required Degree Field": "Design",
        "Years of Experience": 3,
        "Hard Skills": ["GIS Software", "Zoning Regulations", "Community Engagement"],
        "Soft Skills": ["Analytical Thinking", "Communication", "Collaboration"]
    },
    {
        "Job ID": 118,
        "Job Title": "Carpenter",
        "Industry": "Construction and Engineering",
        "Required Education": "None",
        "Required Degree Field": "None",
        "Years of Experience": 2,
        "Hard Skills": ["Woodworking", "Blueprint Reading", "Framing"],
        "Soft Skills": ["Attention to Detail", "Physical Stamina", "Teamwork"]
    },
    {
        "Job ID": 119,
        "Job Title": "Electrician",
        "Industry": "Construction and Engineering",
        "Required Education": "None",
        "Required Degree Field": "None",
        "Years of Experience": 3,
        "Hard Skills": ["Electrical Wiring", "Troubleshooting", "National Electrical Code"],
        "Soft Skills": ["Problem-Solving", "Safety Awareness", "Manual Dexterity"]
    },
    {
        "Job ID": 120,
        "Job Title": "Plumber",
        "Industry": "Construction and Engineering",
        "Required Education": "None",
        "Required Degree Field": "None",
        "Years of Experience": 2,
        "Hard Skills": ["Pipe Installation", "Leak Repair", "Plumbing Codes"],
        "Soft Skills": ["Problem-Solving", "Physical Stamina", "Customer Service"]
    },
    {
        "Job ID": 121,
        "Job Title": "Welder",
        "Industry": "Construction and Engineering",
        "Required Education": "None",
        "Required Degree Field": "None",
        "Years of Experience": 1,
        "Hard Skills": ["Arc Welding", "Metal Fabrication", "Blueprint Reading"],
        "Soft Skills": ["Attention to Detail", "Physical Stamina", "Teamwork"]
    },
    {
        "Job ID": 122,
        "Job Title": "Heavy Equipment Operator",
        "Industry": "Construction and Engineering",
        "Required Education": "None",
        "Required Degree Field": "None",
        "Years of Experience": 2,
        "Hard Skills": ["Equipment Maintenance", "Safety Protocols", "Site Preparation"],
        "Soft Skills": ["Coordination", "Attention to Detail", "Reliability"]
    },
    {
        "Job ID": 123,
        "Job Title": "Manufacturing Engineer",
        "Industry": "Construction and Engineering",
        "Required Education": "Bachelor",
        "Required Degree Field": "Engineering",
        "Years of Experience": 3,
        "Hard Skills": ["Process Improvement", "Lean Manufacturing", "CAD/CAM"],
        "Soft Skills": ["Problem-Solving", "Innovation", "Teamwork"]
    },
    {
        "Job ID": 124,
        "Job Title": "Quality Assurance Engineer",
        "Industry": "Construction and Engineering",
        "Required Education": "Bachelor",
        "Required Degree Field": "Engineering",
        "Years of Experience": 2,
        "Hard Skills": ["Testing Protocols", "Statistical Analysis", "Quality Standards"],
        "Soft Skills": ["Attention to Detail", "Analytical Thinking", "Communication"]
    },
    {
        "Job ID": 125,
        "Job Title": "Factory Supervisor",
        "Industry": "Construction and Engineering",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 4,
        "Hard Skills": ["Production Scheduling", "Inventory Management", "Safety Compliance"],
        "Soft Skills": ["Leadership", "Decision-Making", "Conflict Resolution"]
    },
    {
        "Job ID": 126,
        "Job Title": "Environmental Scientist",
        "Industry": "Environmental and Sustainability",
        "Required Education": "Bachelor",
        "Required Degree Field": "Science",
        "Years of Experience": 3,
        "Hard Skills": ["Environmental Impact Assessment", "GIS", "Data Analysis"],
        "Soft Skills": ["Critical Thinking", "Communication", "Attention to Detail"]
    },
    {
        "Job ID": 127,
        "Job Title": "Sustainability Specialist",
        "Industry": "Environmental and Sustainability",
        "Required Education": "Master",
        "Required Degree Field": "Science",
        "Years of Experience": 4,
        "Hard Skills": ["Sustainability Reporting", "Life Cycle Assessment", "Carbon Footprinting"],
        "Soft Skills": ["Strategic Planning", "Stakeholder Engagement", "Problem-Solving"]
    },
    {
        "Job ID": 128,
        "Job Title": "Renewable Energy Engineer",
        "Industry": "Environmental and Sustainability",
        "Required Education": "Bachelor",
        "Required Degree Field": "Engineering",
        "Years of Experience": 2,
        "Hard Skills": ["Solar Energy Systems", "Wind Turbine Design", "Energy Modeling"],
        "Soft Skills": ["Innovation", "Teamwork", "Project Management"]
    },
    {
        "Job ID": 129,
        "Job Title": "Marine Biologist",
        "Industry": "Science and Research",
        "Required Education": "Master",
        "Required Degree Field": "Science",
        "Years of Experience": 3,
        "Hard Skills": ["Marine Ecosystems Analysis", "Field Sampling", "Statistical Modeling"],
        "Soft Skills": ["Patience", "Observational Skills", "Collaboration"]
    },
    {
        "Job ID": 130,
        "Job Title": "Climate Change Analyst",
        "Industry": "Environmental and Sustainability",
        "Required Education": "Master",
        "Required Degree Field": "Science",
        "Years of Experience": 4,
        "Hard Skills": ["Climate Modeling", "Policy Analysis", "Data Visualization"],
        "Soft Skills": ["Analytical Thinking", "Communication", "Adaptability"]
    },
    {
        "Job ID": 131,
        "Job Title": "Water Resource Specialist",
        "Industry": "Environmental and Sustainability",
        "Required Education": "Bachelor",
        "Required Degree Field": "Engineering",
        "Years of Experience": 3,
        "Hard Skills": ["Hydrological Modeling", "Water Quality Analysis", "GIS"],
        "Soft Skills": ["Problem-Solving", "Teamwork", "Attention to Detail"]
    },
    {
        "Job ID": 132,
        "Job Title": "Ecologist",
        "Industry": "Science and Research",
        "Required Education": "Bachelor",
        "Required Degree Field": "Science",
        "Years of Experience": 2,
        "Hard Skills": ["Ecosystem Assessment", "Field Research", "Species Identification"],
        "Soft Skills": ["Patience", "Observation", "Communication"]
    },
    {
        "Job ID": 133,
        "Job Title": "Agricultural Scientist",
        "Industry": "Science and Research",
        "Required Education": "Master",
        "Required Degree Field": "Science",
        "Years of Experience": 3,
        "Hard Skills": ["Crop Science", "Soil Analysis", "Agricultural Biotechnology"],
        "Soft Skills": ["Research Skills", "Problem-Solving", "Collaboration"]
    },
    {
        "Job ID": 134,
        "Job Title": "Wildlife Conservationist",
        "Industry": "Environmental and Sustainability",
        "Required Education": "Bachelor",
        "Required Degree Field": "Science",
        "Years of Experience": 2,
        "Hard Skills": ["Wildlife Tracking", "Habitat Restoration", "Conservation Planning"],
        "Soft Skills": ["Passion for Wildlife", "Patience", "Teamwork"]
    },
    {
        "Job ID": 135,
        "Job Title": "Environmental Policy Analyst",
        "Industry": "Law and Government",
        "Required Education": "Master",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 4,
        "Hard Skills": ["Policy Analysis", "Regulatory Compliance", "Stakeholder Consultation"],
        "Soft Skills": ["Critical Thinking", "Communication", "Negotiation"]
    },
    {
        "Job ID": 136,
        "Job Title": "Hotel Manager",
        "Industry": "Travel, Hospitality and Tourism",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 5,
        "Hard Skills": ["Hospitality Management", "Revenue Management", "Customer Service"],
        "Soft Skills": ["Leadership", "Communication", "Problem-Solving"]
    },
    {
        "Job ID": 137,
        "Job Title": "Travel Agent",
        "Industry": "Travel, Hospitality and Tourism",
        "Required Education": "None",
        "Required Degree Field": "Business",
        "Years of Experience": 1,
        "Hard Skills": ["Travel Booking Systems", "Itinerary Planning", "Customer Service"],
        "Soft Skills": ["Communication", "Sales Skills", "Attention to Detail"]
    },
    {
        "Job ID": 138,
        "Job Title": "Tour Guide",
        "Industry": "Travel, Hospitality and Tourism",
        "Required Education": "None",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 0,
        "Hard Skills": ["Local Knowledge", "Multilingual Skills", "First Aid"],
        "Soft Skills": ["Public Speaking", "Enthusiasm", "Interpersonal Skills"]
    },
    {
        "Job ID": 139,
        "Job Title": "Event Planner",
        "Industry": "Travel, Hospitality and Tourism",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 3,
        "Hard Skills": ["Event Coordination", "Vendor Management", "Budgeting"],
        "Soft Skills": ["Organization", "Creativity", "Time Management"]
    },
    {
        "Job ID": 140,
        "Job Title": "Flight Attendant",
        "Industry": "Travel, Hospitality and Tourism",
        "Required Education": "None",
        "Required Degree Field": "Business",
        "Years of Experience": 1,
        "Hard Skills": ["Safety Procedures", "Customer Service", "First Aid"],
        "Soft Skills": ["Communication", "Patience", "Teamwork"]
    },
    {
        "Job ID": 141,
        "Job Title": "Pilot",
        "Industry": "Transportation and Logistics",
        "Required Education": "Bachelor",
        "Required Degree Field": "Engineering",
        "Years of Experience": 5,
        "Hard Skills": ["Flight Operations", "Navigation Systems", "Aircraft Systems"],
        "Soft Skills": ["Decision-Making", "Leadership", "Calm Under Pressure"]
    },
    {
        "Job ID": 142,
        "Job Title": "Cruise Director",
        "Industry": "Travel, Hospitality and Tourism",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 4,
        "Hard Skills": ["Entertainment Coordination", "Guest Relations", "Event Management"],
        "Soft Skills": ["Leadership", "Communication", "Creativity"]
    },
    {
        "Job ID": 143,
        "Job Title": "Resort Manager",
        "Industry": "Travel, Hospitality and Tourism",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 5,
        "Hard Skills": ["Resort Operations", "Hospitality Management", "Revenue Optimization"],
        "Soft Skills": ["Leadership", "Customer Service", "Problem-Solving"]
    },
    {
        "Job ID": 144,
        "Job Title": "Concierge",
        "Industry": "Travel, Hospitality and Tourism",
        "Required Education": "None",
        "Required Degree Field": "Business",
        "Years of Experience": 2,
        "Hard Skills": ["Guest Services", "Local Knowledge", "Reservation Systems"],
        "Soft Skills": ["Communication", "Problem-Solving", "Attention to Detail"]
    },
    {
        "Job ID": 145,
        "Job Title": "Restaurant Manager",
        "Industry": "Travel, Hospitality and Tourism",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 3,
        "Hard Skills": ["Restaurant Operations", "Inventory Management", "Staff Scheduling"],
        "Soft Skills": ["Leadership", "Customer Service", "Teamwork"]
    },
    {
        "Job ID": 146,
        "Job Title": "Chemist",
        "Industry": "Science and Research",
        "Required Education": "Bachelor",
        "Required Degree Field": "Science",
        "Years of Experience": 2,
        "Hard Skills": ["Chemical Analysis", "Laboratory Techniques", "Data Interpretation"],
        "Soft Skills": ["Attention to Detail", "Analytical Thinking", "Problem-Solving"]
    },
    {
        "Job ID": 147,
        "Job Title": "Physicist",
        "Industry": "Science and Research",
        "Required Education": "Master",
        "Required Degree Field": "Science",
        "Years of Experience": 3,
        "Hard Skills": ["Theoretical Modeling", "Data Analysis", "Computational Physics"],
        "Soft Skills": ["Critical Thinking", "Curiosity", "Patience"]
    },
    {
        "Job ID": 148,
        "Job Title": "Biologist",
        "Industry": "Science and Research",
        "Required Education": "Bachelor",
        "Required Degree Field": "Science",
        "Years of Experience": 2,
        "Hard Skills": ["Microscopy", "DNA Analysis", "Statistical Methods"],
        "Soft Skills": ["Observation", "Research Skills", "Collaboration"]
    },
    {
        "Job ID": 149,
        "Job Title": "Astronomer",
        "Industry": "Science and Research",
        "Required Education": "Master",
        "Required Degree Field": "Science",
        "Years of Experience": 4,
        "Hard Skills": ["Telescope Operation", "Astrophysical Modeling", "Data Analysis"],
        "Soft Skills": ["Patience", "Analytical Thinking", "Problem-Solving"]
    },
    {
        "Job ID": 150,
        "Job Title": "Mathematician",
        "Industry": "Science and Research",
        "Required Education": "Master",
        "Required Degree Field": "Science",
        "Years of Experience": 3,
        "Hard Skills": ["Mathematical Modeling", "Statistical Analysis", "Algorithm Development"],
        "Soft Skills": ["Logical Thinking", "Creativity", "Attention to Detail"]
    },
    {
        "Job ID": 151,
        "Job Title": "Statistician",
        "Industry": "Science and Research",
        "Required Education": "Master",
        "Required Degree Field": "Science",
        "Years of Experience": 3,
        "Hard Skills": ["Statistical Software", "Data Analysis", "Experimental Design"],
        "Soft Skills": ["Analytical Thinking", "Problem-Solving", "Communication"]
    },
    {
        "Job ID": 152,
        "Job Title": "Geneticist",
        "Industry": "Science and Research",
        "Required Education": "Master",
        "Required Degree Field": "Science",
        "Years of Experience": 4,
        "Hard Skills": ["Genomic Sequencing", "PCR", "Bioinformatics"],
        "Soft Skills": ["Research Skills", "Attention to Detail", "Collaboration"]
    },
    {
        "Job ID": 153,
        "Job Title": "Biomedical Engineer",
        "Industry": "Healthcare",
        "Required Education": "Bachelor",
        "Required Degree Field": "Engineering",
        "Years of Experience": 3,
        "Hard Skills": ["Medical Device Design", "Biomechanics", "Signal Processing"],
        "Soft Skills": ["Problem-Solving", "Teamwork", "Innovation"]
    },
    {
        "Job ID": 154,
        "Job Title": "Materials Scientist",
        "Industry": "Science and Research",
        "Required Education": "Master",
        "Required Degree Field": "Science",
        "Years of Experience": 4,
        "Hard Skills": ["Materials Characterization", "Polymer Science", "Nanotechnology"],
        "Soft Skills": ["Research Skills", "Analytical Thinking", "Creativity"]
    },
    {
        "Job ID": 155,
        "Job Title": "Forensic Scientist",
        "Industry": "Science and Research",
        "Required Education": "Bachelor",
        "Required Degree Field": "Science",
        "Years of Experience": 3,
        "Hard Skills": ["DNA Analysis", "Toxicology", "Crime Scene Investigation"],
        "Soft Skills": ["Attention to Detail", "Critical Thinking", "Communication"]
    },
    {
        "Job ID": 156,
        "Job Title": "Retail Store Manager",
        "Industry": "Retail and Sales",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 4,
        "Hard Skills": ["Inventory Management", "Sales Analysis", "Staff Training"],
        "Soft Skills": ["Leadership", "Customer Service", "Problem-Solving"]
    },
    {
        "Job ID": 157,
        "Job Title": "Cashier",
        "Industry": "Retail and Sales",
        "Required Education": "None",
        "Required Degree Field": "Business",
        "Years of Experience": 0,
        "Hard Skills": ["Point of Sale Systems", "Cash Handling", "Product Knowledge"],
        "Soft Skills": ["Customer Service", "Attention to Detail", "Friendliness"]
    },
    {
        "Job ID": 158,
        "Job Title": "Visual Merchandiser",
        "Industry": "Retail and Sales",
        "Required Education": "Bachelor",
        "Required Degree Field": "Design",
        "Years of Experience": 2,
        "Hard Skills": ["Store Layout Design", "Display Creation", "Trend Analysis"],
        "Soft Skills": ["Creativity", "Attention to Detail", "Communication"]
    },
    {
        "Job ID": 159,
        "Job Title": "Customer Service Representative",
        "Industry": "Retail and Sales",
        "Required Education": "None",
        "Required Degree Field": "Business",
        "Years of Experience": 1,
        "Hard Skills": ["CRM Software", "Order Processing", "Product Knowledge"],
        "Soft Skills": ["Communication", "Empathy", "Problem-Solving"]
    },
    {
        "Job ID": 160,
        "Job Title": "E-commerce Manager",
        "Industry": "Retail and Sales",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 3,
        "Hard Skills": ["Online Marketing", "SEO", "Web Analytics"],
        "Soft Skills": ["Strategic Thinking", "Communication", "Adaptability"]
    },
    {
        "Job ID": 161,
        "Job Title": "Sales Associate",
        "Industry": "Retail and Sales",
        "Required Education": "None",
        "Required Degree Field": "Business",
        "Years of Experience": 1,
        "Hard Skills": ["Product Knowledge", "Sales Techniques", "Customer Relationship Management"],
        "Soft Skills": ["Communication", "Persuasion", "Friendliness"]
    },
    {
        "Job ID": 162,
        "Job Title": "Account Manager",
        "Industry": "Retail and Sales",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 3,
        "Hard Skills": ["Client Relationship Management", "Sales Forecasting", "Contract Negotiation"],
        "Soft Skills": ["Communication", "Negotiation", "Problem-Solving"]
    },
    {
        "Job ID": 163,
        "Job Title": "Key Account Executive",
        "Industry": "Retail and Sales",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 4,
        "Hard Skills": ["Account Strategy", "Revenue Growth", "Market Analysis"],
        "Soft Skills": ["Relationship Building", "Negotiation", "Strategic Thinking"]
    },
    {
        "Job ID": 164,
        "Job Title": "Retail Buyer",
        "Industry": "Retail and Sales",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 3,
        "Hard Skills": ["Vendor Negotiation", "Inventory Planning", "Trend Forecasting"],
        "Soft Skills": ["Analytical Thinking", "Decision-Making", "Communication"]
    },
    {
        "Job ID": 165,
        "Job Title": "Merchandise Planner",
        "Industry": "Retail and Sales",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 3,
        "Hard Skills": ["Demand Forecasting", "Inventory Optimization", "Sales Analysis"],
        "Soft Skills": ["Analytical Skills", "Attention to Detail", "Problem-Solving"]
    },
    {
        "Job ID": 166,
        "Job Title": "Public Policy Analyst",
        "Industry": "Law and Government",
        "Required Education": "Master",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 4,
        "Hard Skills": ["Policy Research", "Legislative Analysis", "Stakeholder Engagement"],
        "Soft Skills": ["Critical Thinking", "Communication", "Negotiation"]
    },
    {
        "Job ID": 167,
        "Job Title": "Police Officer",
        "Industry": "Law and Government",
        "Required Education": "None",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 2,
        "Hard Skills": ["Law Enforcement", "Criminal Investigation", "First Aid"],
        "Soft Skills": ["Communication", "Decision-Making", "Physical Fitness"]
    },
    {
        "Job ID": 168,
        "Job Title": "Firefighter",
        "Industry": "Law and Government",
        "Required Education": "None",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 2,
        "Hard Skills": ["Fire Suppression", "Emergency Medical Response", "Hazardous Materials Handling"],
        "Soft Skills": ["Bravery", "Teamwork", "Physical Fitness"]
    },
    {
        "Job ID": 169,
        "Job Title": "Diplomat",
        "Industry": "Law and Government",
        "Required Education": "Master",
        "Required Degree Field": "Arts and Social Sciences"
    },
    {
        "Job ID": 170,
        "Job Title": "Immigration Officer",
        "Industry": "Law and Government",
        "Required Education": "Bachelor",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 3,
        "Hard Skills": ["Legal Documentation", "Immigration Laws", "Case Management"],
        "Soft Skills": ["Communication", "Empathy", "Attention to Detail"]
    },
    {
        "Job ID": 171,
        "Job Title": "Judge",
        "Industry": "Law and Government",
        "Required Education": "Master",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 5,
        "Hard Skills": ["Legal Research", "Judicial Procedures", "Courtroom Management"],
        "Soft Skills": ["Decision-Making", "Integrity", "Leadership"]
    },
    {
        "Job ID": 172,
        "Job Title": "Paralegal",
        "Industry": "Law and Government",
        "Required Education": "Bachelor",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 2,
        "Hard Skills": ["Legal Writing", "Document Drafting", "Case Preparation"],
        "Soft Skills": ["Organization", "Time Management", "Teamwork"]
    },
    {
        "Job ID": 173,
        "Job Title": "Legal Assistant",
        "Industry": "Law and Government",
        "Required Education": "Bachelor",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 1,
        "Hard Skills": ["Legal Research", "Document Management", "Scheduling"],
        "Soft Skills": ["Communication", "Attention to Detail", "Adaptability"]
    },
    {
        "Job ID": 174,
        "Job Title": "Public Defender",
        "Industry": "Law and Government",
        "Required Education": "Master",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 4,
        "Hard Skills": ["Criminal Law", "Trial Advocacy", "Client Representation"],
        "Soft Skills": ["Empathy", "Public Speaking", "Resilience"]
    },
    {
        "Job ID": 175,
        "Job Title": "Urban Policy Planner",
        "Industry": "Law and Government",
        "Required Education": "Master",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 3,
        "Hard Skills": ["Policy Analysis", "Urban Development", "GIS Mapping"],
        "Soft Skills": ["Critical Thinking", "Collaboration", "Negotiation"]
    },
    {
        "Job ID": 176,
        "Job Title": "Supply Chain Manager",
        "Industry": "Transportation and Logistics",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 4,
        "Hard Skills": ["Inventory Management", "Logistics Software", "Supplier Negotiation"],
        "Soft Skills": ["Leadership", "Problem-Solving", "Strategic Thinking"]
    },
    {
        "Job ID": 177,
        "Job Title": "Logistics Coordinator",
        "Industry": "Transportation and Logistics",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 2,
        "Hard Skills": ["Shipping Coordination", "Route Planning", "ERP Systems"],
        "Soft Skills": ["Organization", "Communication", "Multitasking"]
    },
    {
        "Job ID": 178,
        "Job Title": "Transportation Planner",
        "Industry": "Transportation and Logistics",
        "Required Education": "Bachelor",
        "Required Degree Field": "Engineering",
        "Years of Experience": 3,
        "Hard Skills": ["Traffic Analysis", "GIS Software", "Transport Modeling"],
        "Soft Skills": ["Analytical Thinking", "Teamwork", "Project Management"]
    },
    {
        "Job ID": 179,
        "Job Title": "Warehouse Manager",
        "Industry": "Transportation and Logistics",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 3,
        "Hard Skills": ["Inventory Control", "Warehouse Software", "Safety Compliance"],
        "Soft Skills": ["Leadership", "Time Management", "Problem-Solving"]
    },
    {
        "Job ID": 180,
        "Job Title": "Freight Broker",
        "Industry": "Transportation and Logistics",
        "Required Education": "None",
        "Required Degree Field": "Business",
        "Years of Experience": 1,
        "Hard Skills": ["Freight Negotiation", "Load Board Management", "Shipping Regulations"],
        "Soft Skills": ["Communication", "Networking", "Persuasion"]
    },
    {
        "Job ID": 181,
        "Job Title": "Truck Driver",
        "Industry": "Transportation and Logistics",
        "Required Education": "None",
        "Required Degree Field": "None",
        "Years of Experience": 0,
        "Hard Skills": ["Vehicle Operation", "Route Navigation", "Cargo Handling"],
        "Soft Skills": ["Time Management", "Patience", "Safety Awareness"]
    },
    {
        "Job ID": 182,
        "Job Title": "Airline Operations Manager",
        "Industry": "Transportation and Logistics",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 4,
        "Hard Skills": ["Flight Scheduling", "Regulatory Compliance", "Operations Software"],
        "Soft Skills": ["Leadership", "Decision-Making", "Crisis Management"]
    },
    {
        "Job ID": 183,
        "Job Title": "Port Manager",
        "Industry": "Transportation and Logistics",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 5,
        "Hard Skills": ["Port Operations", "Maritime Regulations", "Cargo Handling"],
        "Soft Skills": ["Leadership", "Strategic Planning", "Stakeholder Management"]
    },
    {
        "Job ID": 184,
        "Job Title": "Inventory Manager",
        "Industry": "Transportation and Logistics",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 3,
        "Hard Skills": ["Stock Control", "Demand Forecasting", "ERP Systems"],
        "Soft Skills": ["Attention to Detail", "Analytical Thinking", "Organization"]
    },
    {
        "Job ID": 185,
        "Job Title": "Procurement Officer",
        "Industry": "Transportation and Logistics",
        "Required Education": "Bachelor",
        "Required Degree Field": "Business",
        "Years of Experience": 2,
        "Hard Skills": ["Vendor Management", "Contract Negotiation", "Purchase Order Systems"],
        "Soft Skills": ["Negotiation", "Communication", "Cost Awareness"]
    },
    {
        "Job ID": 186,
        "Job Title": "AI Product Manager",
        "Industry": "Technology",
        "Required Education": "Master",
        "Required Degree Field": "Computing",
        "Years of Experience": 4,
        "Hard Skills": ["Machine Learning", "Product Roadmapping", "Agile Development"],
        "Soft Skills": ["Leadership", "Strategic Thinking", "Stakeholder Management"]
    },
    {
        "Job ID": 187,
        "Job Title": "Prompt Engineer (for AI)",
        "Industry": "Technology",
        "Required Education": "Bachelor",
        "Required Degree Field": "Computing",
        "Years of Experience": 2,
        "Hard Skills": ["Natural Language Processing", "AI Model Fine-Tuning", "Python"],
        "Soft Skills": ["Creativity", "Problem-Solving", "Attention to Detail"]
    },
    {
        "Job ID": 188,
        "Job Title": "Data Privacy Officer",
        "Industry": "Technology",
        "Required Education": "Master",
        "Required Degree Field": "Information Systems",
        "Years of Experience": 5,
        "Hard Skills": ["GDPR Compliance", "Risk Assessment", "Data Encryption"],
        "Soft Skills": ["Ethical Judgment", "Communication", "Policy Development"]
    },
    {
        "Job ID": 189,
        "Job Title": "Robotics Engineer",
        "Industry": "Technology",
        "Required Education": "Master",
        "Required Degree Field": "Engineering",
        "Years of Experience": 3,
        "Hard Skills": ["Robotics Programming", "CAD Design", "Control Systems"],
        "Soft Skills": ["Innovation", "Collaboration", "Problem-Solving"]
    },
    {
        "Job ID": 190,
        "Job Title": "VR/AR Developer",
        "Industry": "Technology",
        "Required Education": "Bachelor",
        "Required Degree Field": "Computing",
        "Years of Experience": 2,
        "Hard Skills": ["Unity 3D", "3D Modeling", "C# Programming"],
        "Soft Skills": ["Creativity", "Teamwork", "User-Centric Thinking"]
    },
    {
        "Job ID": 191,
        "Job Title": "Quantum Computing Researcher",
        "Industry": "Science and Research",
        "Required Education": "Master",
        "Required Degree Field": "Science",
        "Years of Experience": 4,
        "Hard Skills": ["Quantum Algorithms", "Linear Algebra", "Qiskit"],
        "Soft Skills": ["Analytical Thinking", "Curiosity", "Persistence"]
    },
    {
        "Job ID": 192,
        "Job Title": "Drone Operator",
        "Industry": "Technology",
        "Required Education": "None",
        "Required Degree Field": "None",
        "Years of Experience": 1,
        "Hard Skills": ["Drone Piloting", "Aerial Photography", "FAA Regulations"],
        "Soft Skills": ["Precision", "Safety Awareness", "Adaptability"]
    },
    {
        "Job ID": 193,
        "Job Title": "Space Tourism Guide",
        "Industry": "Travel, Hospitality and Tourism",
        "Required Education": "Bachelor",
        "Required Degree Field": "Arts and Social Sciences",
        "Years of Experience": 2,
        "Hard Skills": ["Customer Service", "Safety Protocols", "Tour Coordination"],
        "Soft Skills": ["Communication", "Enthusiasm", "Public Speaking"]
    },
    {
        "Job ID": 194,
        "Job Title": "Smart Home Technician",
        "Industry": "Technology",
        "Required Education": "None",
        "Required Degree Field": "Engineering",
        "Years of Experience": 1,
        "Hard Skills": ["IoT Device Installation", "Network Configuration", "Troubleshooting"],
        "Soft Skills": ["Customer Service", "Problem-Solving", "Technical Aptitude"]
    },
    {
        "Job ID": 195,
        "Job Title": "Climate Tech Entrepreneur",
        "Industry": "Environmental and Sustainability",
        "Required Education": "Master",
        "Required Degree Field": "Business",
        "Years of Experience": 5,
        "Hard Skills": ["Sustainable Technologies", "Business Development", "Fundraising"],
        "Soft Skills": ["Innovation", "Risk-Taking", "Leadership"]
    },
    {
        "Job ID": 196,
        "Job Title": "Blockchain Product Manager",
        "Industry": "Technology",
        "Required Education": "Master",
        "Required Degree Field": "Computing",
        "Years of Experience": 4,
        "Hard Skills": ["Blockchain Protocols", "Smart Contracts", "Decentralized Applications"],
        "Soft Skills": ["Strategic Vision", "Collaboration", "Market Analysis"]
    },
    {
        "Job ID": 197,
        "Job Title": "Autonomous Vehicle Engineer",
        "Industry": "Technology",
        "Required Education": "Master",
        "Required Degree Field": "Engineering",
        "Years of Experience": 3,
        "Hard Skills": ["Machine Learning", "Sensor Fusion", "ROS (Robot Operating System)"],
        "Soft Skills": ["Problem-Solving", "Teamwork", "Innovation"]
    },
    {
        "Job ID": 198,
        "Job Title": "IoT (Internet of Things) Engineer",
        "Industry": "Technology",
        "Required Education": "Bachelor",
        "Required Degree Field": "Engineering",
        "Years of Experience": 2,
        "Hard Skills": ["Embedded Systems", "Wireless Protocols", "Cloud Integration"],
        "Soft Skills": ["Analytical Thinking", "Collaboration", "Creativity"]
    },
    {
        "Job ID": 199,
        "Job Title": "Digital Twin Engineer",
        "Industry": "Technology",
        "Required Education": "Master",
        "Required Degree Field": "Engineering",
        "Years of Experience": 3,
        "Hard Skills": ["3D Modeling", "Simulation Software", "Data Analytics"],
        "Soft Skills": ["Attention to Detail", "Problem-Solving", "Interdisciplinary Collaboration"]
    },
    {
        "Job ID": 200,
        "Job Title": "Ethical Hacker",
        "Industry": "Technology",
        "Required Education": "Bachelor",
        "Required Degree Field": "Computing",
        "Years of Experience": 3,
        "Hard Skills": ["Penetration Testing", "Network Security", "Vulnerability Assessment"],
        "Soft Skills": ["Curiosity", "Ethical Judgment", "Problem-Solving"]
    }
]

In [23]:
df = pd.read_csv('job_data.csv')
df

,Job ID,Job Title,Industry,Required Education,Required Degree Field,Years of Experience,Hard Skills,Soft Skills
0,1,Software Engineer,Technology,Bachelor,Computing,2,"['Python', 'Software Development', 'Version Co...","['Problem-Solving', 'Teamwork']"
1,2,Frontend Developer,Technology,Bachelor,Computing,1,"['JavaScript', 'HTML/CSS', 'React']","['Creativity', 'Attention to Detail']"
2,3,Backend Developer,Technology,Bachelor,Computing,2,"['Java', 'SQL', 'API Development']","['Logical Thinking', 'Collaboration']"
3,4,Full Stack Developer,Technology,Bachelor,Computing,3,"['JavaScript', 'Python', 'Node.js']","['Adaptability', 'Communication']"
4,5,DevOps Engineer,Technology,Bachelor,Computing,3,"['Docker', 'Kubernetes', 'CI/CD']","['Problem-Solving', 'Teamwork']"
...,...,...,...,...,...,...,...,...
195,196,Blockchain Product Manager,Technology,Master,Computing,4,"['Blockchain Protocols', 'Smart Contracts', 'D...","['Strategic Vision', 'Collaboration', 'Market ..."
196,197,Autonomous Vehicle Engineer,Technology,Master,Engineering,3,"['Machine Learning', 'Sensor Fusion', 'ROS (Ro...","['Problem-Solving', 'Teamwork', 'Innovation']"
197,198,IoT (Internet of Things) Engineer,Technology,Bachelor,Engineering,2,"['Embedded Systems', 'Wireless Protocols', 'Cl...","['Analytical Thinking', 'Collaboration', 'Crea..."
198,199,Digital Twin Engineer,Technology,Master,Engineering,3,"['3D Modeling', 'Simulation Software', 'Data A...","['Attention to Detail', 'Problem-Solving', 'In..."


### Create a resume parser that converts student resume into a dataframe

In [42]:
# Using pdfminer.six to extract text from resume
text = extract_text("Tay Zhi Wen Jeremiah CV.pdf")
print(text)



Tay Zhi Wen Jeremiah 

HP: +65 86125565      Email: tayjeremiah2000@gmail.com/ e0726076@u.nus.edu 
GitHub: https://github.com/jeremiah-tay 

EDUCATION 

Aug 2021 - Dec 2025 

National University of Singapore 
Bachelor of Science in Data Science and Analytics (Hons) 
•  Grades: Honors with Distinction 
• 

Coursework:  Convex  Optimization,  Data  Science  in  Practice,  Data  Structures  and  Algorithms,  Data  Visualization,  Deep  Learning,  Efficient 
Algorithms  and  Data  Structures  2,  Machine  Learning,  Modelling  and  Simulation,  Numerical  Computation,  Regression  Analysis,  Statistical 
Computing in R, Statistical Learning, Database Technology and management 

WORK EXPERIENCE 

Sep 2024 – Jan 2025 
AgileAlgo 
Data Science Intern 
Python, LangChain, RAG, LLM, Power BI 
•  Designed and implemented a back end large language model (LLM) pipeline using Python, delivering a scalable architecture for the company’s 

• 

• 

Virtual System Implementer Platform. 
Enhanced  LLM  pe

In [51]:
prompt = f"""
You are a professional headhunting specialist tasked with extracting structured information from a candidate's resume.

Please extract the following details accurately from the resume text provided below:

- Name (full name of the candidate)
- Education Level (e.g., Bachelor, Master, PhD)
- Degree Field (e.g., Data Science, Computer Science, Economics)
- University (name of the institution)
- GPA (if available)
- Work Experience (in total years)
- Hard Skills (list of technical or domain-specific skills)
- Soft Skills (list of interpersonal or transferable skills)

Resume:
{text}

Return your response **strictly** in the following format without any additional explanation:

Name: 
Education Level:
Degree Field:
University:
GPA:
Work Experience (years):
Hard Skills:
Soft Skills:
"""

response = client.chat.completions.create(
    model = "deepseek-chat",
    messages = [
        {"role": "system", "content": "You are an extremely experienced head hunting specialist, that is experienced with reading resumes."},
        {"role": "user", "content": prompt},
    ],
    stream = False
)

print(response.choices[0].message.content)

Name: Tay Zhi Wen Jeremiah  
Education Level: Bachelor  
Degree Field: Data Science and Analytics (Hons)  
University: National University of Singapore  
GPA: Honors with Distinction  
Work Experience (years): 1.5  
Hard Skills: Python, LangChain, RAG, LLM, Power BI, Regex, JavaScript, PostgreSQL, Git, GitHub, Docker, ChromaDB, OCRmyPDF, PyPDF2, R, Java, MATLAB, MySQL, SparkSQL, Scala, Cypher, Tableau, Jupyter Notebook, Keras, TensorFlow, Scikit-Learn, Data Cleaning, Data Modelling, Exploratory Data Analysis, Data Visualization, Predictive Analysis, Linear Programming, Convex Optimization, Microsoft Word, Excel, PowerPoint  
Soft Skills: Collaboration, Knowledge Sharing, Problem Solving


In [60]:
column_names = ['Name', 'Education Level', 'Degree Field', 'University', 'GPA',
                'Work Experience', 'Hard Skills', 'Soft Skills']
student_df = pd.DataFrame(columns=column_names)

new_student = {}

for col in column_names:
    pattern = rf"{col}:\s*?(.*)"
    match = re.search(pattern, response.choices[0].message.content)
    if match:
        value = match.group(1).strip()
        new_student[col] = value
    else:
        new_student[col] = ""

new_student = pd.DataFrame([new_student])
student_df = pd.concat([student_df, new_student], ignore_index=True)
student_df

,Name,Education Level,Degree Field,University,GPA,Work Experience,Hard Skills,Soft Skills
0,Tay Zhi Wen Jeremiah,Bachelor,Data Science and Analytics (Hons),National University of Singapore,Honors with Distinction,,"Python, LangChain, RAG, LLM, Power BI, Regex, ...","Collaboration, Knowledge Sharing, Problem Solving"


In [41]:
# Using pdfminer.six to extract text from resume
text = extract_text("Tay Zhi Wen Jeremiah CV.pdf")
print(text)

nlp = spacy.load('en_core_web_sm')

# Process the resume text
doc = nlp(text)

# Extract entities
entity_dict = defaultdict(list)
for ent in doc.ents:
    entity_dict[ent.label_].append(ent.text)

# Optional: convert lists to strings (you can tweak this depending on your use case)
for key in entity_dict:
    if key not in ['HARD_SKILL', 'SOFT_SKILL']:
        entity_dict[key] = entity_dict[key][0] if len(entity_dict[key]) == 1 else ", ".join(entity_dict[key])

columns = ["Name", "Education Level", "Degree Field", "University", 
           "GPA", "Years Experience", "Hard Skills", "Soft Skills"]

# Ensure all columns are present
row = {col: entity_dict.get(col, []) for col in columns}

# Create single-row DataFrame
df1 = pd.DataFrame([row])
df1

,Name,Education Level,Degree Field,University,GPA,Years Experience,Hard Skills,Soft Skills
0,[],[],[],[],[],[],[],[]
